# EDA Notes

A first attempt at EDA for the CS109 final project. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV
%matplotlib inline

**Read In Data**

In [2]:
checkins = pd.read_csv('Data/checkin_summary.csv', header=None)
photos = pd.read_csv('Data/photos_summary.csv', header=None)
reviews = pd.read_csv('Data/review_small.csv', header=None)
biz = pd.read_csv('Data/small_businesses.csv')
tips = pd.read_csv('Data/tip_summary.csv', header=None)
users = pd.read_json(path_or_buf='Data/user_small.json', lines=True)

/Users/maiawoluchem/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,10,12,14,20,63,64,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Explore data in its current state

In [ ]:
checkins.shape, photos.shape, reviews.shape, biz.shape, tips.shape, users.shape

In [ ]:
checkins.columns = ['business_id', 'num_checkins'] 
checkins.head()

In [ ]:
photos.columns = ['business_id', 'num_photos'] 
photos.head()

In [3]:
reviews.columns = ['user_id', 'business_id', 'rating', 'date'] 
reviews.head()

,user_id,business_id,rating,date
0,V2exkc2JwEdc8UNUNhT7sg,--6MefnULPED_I942VcFNA,5,2016-08-25
1,j97y1ZAsYABIOcR0KQCy5Q,--6MefnULPED_I942VcFNA,1,2010-12-06
2,8-KBKMJVO-j5YqpHb08j9w,--6MefnULPED_I942VcFNA,4,2015-08-05
3,CxDOIDnH8gp9KXzpBHJYXw,--6MefnULPED_I942VcFNA,2,2010-12-22
4,7Dykd1HolQx8mKPYhYDYSg,--6MefnULPED_I942VcFNA,2,2017-02-07


In [4]:
biz.columns=biz.columns.str.replace('attributes.','')
biz.columns=biz.columns.str.replace('.',"_")
biz.head()

,Ambience_divey,HairSpecializesIn_kids,RestaurantsDelivery,Open24Hours,Ambience_trendy,DogsAllowed,CoatCheck,postal_code,Smoking,DietaryRestrictions_halal,...,RestaurantsReservations,DietaryRestrictions_dairy-free,BestNights_wednesday,DietaryRestrictions_vegan,Ambience_romantic,Music_jukebox,Ambience_upscale,RestaurantsTakeOut,BikeParking,OutdoorSeating
0,False,NaN,False,NaN,False,NaN,False,28105,NaN,NaN,...,False,NaN,NaN,NaN,False,False,False,True,True,False
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28164,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,True,NaN,False,NaN,False,NaN,NaN,15224,NaN,NaN,...,False,NaN,NaN,NaN,False,NaN,False,True,True,False
3,NaN,NaN,False,NaN,NaN,NaN,NaN,L5W 0E6,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True
4,False,NaN,False,NaN,False,NaN,NaN,53704,NaN,NaN,...,True,NaN,NaN,NaN,False,NaN,False,True,True,False


In [5]:
#Merge reviews and business
full_biz = pd.merge(reviews, biz, on='business_id', how='inner')

In [ ]:
#full_biz_inner = pd.merge(reviews, biz, on='business_id', how='inner')

In [ ]:
#full_biz_inner.shape, reviews.shape, biz.shape

In [35]:
full_biz.shape, reviews.shape, biz.shape

((2531183, 52), (2927859, 4), (38668, 93))

In [6]:
#Drop where more than 50 percent missing
full_biz = full_biz.dropna(thresh=len(full_biz) - 1265591, axis=1)

In [9]:
columns = list(full_biz.columns)

In [18]:
full_biz.dtypes

user_id                        object
business_id                    object
rating                          int64
date                           object
Ambience_divey                 object
RestaurantsDelivery            object
Ambience_trendy                object
postal_code                    object
city                           object
GoodForKids                    object
Ambience_casual                object
stars                         float64
latitude                      float64
Alcohol                        object
RestaurantsPriceRange2        float64
Ambience_classy                object
RestaurantsTableService        object
GoodForMeal_latenight          object
GoodForMeal_dessert            object
Ambience_touristy              object
BusinessParking_validated      object
GoodForMeal_lunch              object
RestaurantsGoodForGroups       object
categories                     object
GoodForMeal_brunch             object
name                           object
BusinessPark

In [14]:
full_biz[columns[:20]].head()

,user_id,business_id,rating,date,Ambience_divey,RestaurantsDelivery,Ambience_trendy,postal_code,city,GoodForKids,Ambience_casual,stars,latitude,Alcohol,RestaurantsPriceRange2,Ambience_classy,RestaurantsTableService,GoodForMeal_latenight,GoodForMeal_dessert,Ambience_touristy
0,V2exkc2JwEdc8UNUNhT7sg,--6MefnULPED_I942VcFNA,5,2016-08-25,NaN,False,False,L4B 3P7,Richmond Hill,True,True,3.0,43.840905,beer_and_wine,2.0,False,True,False,False,False
1,j97y1ZAsYABIOcR0KQCy5Q,--6MefnULPED_I942VcFNA,1,2010-12-06,NaN,False,False,L4B 3P7,Richmond Hill,True,True,3.0,43.840905,beer_and_wine,2.0,False,True,False,False,False
2,8-KBKMJVO-j5YqpHb08j9w,--6MefnULPED_I942VcFNA,4,2015-08-05,NaN,False,False,L4B 3P7,Richmond Hill,True,True,3.0,43.840905,beer_and_wine,2.0,False,True,False,False,False
3,CxDOIDnH8gp9KXzpBHJYXw,--6MefnULPED_I942VcFNA,2,2010-12-22,NaN,False,False,L4B 3P7,Richmond Hill,True,True,3.0,43.840905,beer_and_wine,2.0,False,True,False,False,False
4,7Dykd1HolQx8mKPYhYDYSg,--6MefnULPED_I942VcFNA,2,2017-02-07,NaN,False,False,L4B 3P7,Richmond Hill,True,True,3.0,43.840905,beer_and_wine,2.0,False,True,False,False,False


In [20]:
full_biz.RestaurantsPriceRange2.unique()

array([  2.,   4.,   1.,  nan,   3.])

In [15]:
full_biz[columns[20:40]].head()

,BusinessParking_validated,GoodForMeal_lunch,RestaurantsGoodForGroups,categories,GoodForMeal_brunch,name,BusinessParking_valet,WheelchairAccessible,Ambience_intimate,Ambience_hipster,BusinessAcceptsCreditCards,is_open,neighborhood,BusinessParking_garage,WiFi,longitude,review_count,GoodForMeal_breakfast,NoiseLevel,state
0,False,True,True,"['Restaurants', 'Chinese']",False,John's Chinese BBQ Restaurant,False,True,False,False,True,1,NaN,False,no,-79.399604,30,False,quiet,ON
1,False,True,True,"['Restaurants', 'Chinese']",False,John's Chinese BBQ Restaurant,False,True,False,False,True,1,NaN,False,no,-79.399604,30,False,quiet,ON
2,False,True,True,"['Restaurants', 'Chinese']",False,John's Chinese BBQ Restaurant,False,True,False,False,True,1,NaN,False,no,-79.399604,30,False,quiet,ON
3,False,True,True,"['Restaurants', 'Chinese']",False,John's Chinese BBQ Restaurant,False,True,False,False,True,1,NaN,False,no,-79.399604,30,False,quiet,ON
4,False,True,True,"['Restaurants', 'Chinese']",False,John's Chinese BBQ Restaurant,False,True,False,False,True,1,NaN,False,no,-79.399604,30,False,quiet,ON


In [25]:
full_biz.RestaurantsAttire.unique()

array(['casual', 'dressy', nan, 'formal'], dtype=object)

In [17]:
full_biz[columns[40:]].head()

,BusinessParking_lot,HasTV,GoodForMeal_dinner,BusinessParking_street,RestaurantsAttire,Caters,RestaurantsReservations,Ambience_romantic,Ambience_upscale,RestaurantsTakeOut,BikeParking,OutdoorSeating
0,True,False,True,False,casual,False,True,False,False,True,False,False
1,True,False,True,False,casual,False,True,False,False,True,False,False
2,True,False,True,False,casual,False,True,False,False,True,False,False
3,True,False,True,False,casual,False,True,False,False,True,False,False
4,True,False,True,False,casual,False,True,False,False,True,False,False


In [26]:
binary_vars = ["Caters","RestaurantsReservations","Ambience_romantic","Ambience_upscale",
               "RestaurantsTakeOut","BikeParking","OutdoorSeating","BusinessParking_lot","HasTV","GoodForMeal_dinner",
               "BusinessParking_street","GoodForMeal_breakfast","BusinessParking_garage","BusinessParking_valet",
               "WheelchairAccessible","Ambience_intimate","Ambience_hipster","BusinessAcceptsCreditCards",
               "GoodForMeal_brunch","Ambience_classy","RestaurantsTableService","GoodForMeal_latenight",
               "GoodForMeal_dessert","Ambience_touristy","BusinessParking_validated","GoodForMeal_lunch",
               "RestaurantsGoodForGroups","GoodForKids","Ambience_casual","Ambience_divey","RestaurantsDelivery",
               "Ambience_trendy"]


In [27]:
text_vars = ["user_id","business_id","postal_code","city","categories",
             "name","neighborhood","state"]

In [29]:
float_vars = ["latitude","longitude","is_open","rating","stars","review_count","RestaurantsPriceRange2"]

In [30]:
cat_vars = ["Alcohol","WiFi","NoiseLevel","RestaurantsAttire"]

In [31]:
date_vars = ['date']

In [34]:
count = len(binary_vars)+len(text_vars)+len(float_vars)+len(cat_vars)+1
count

52

In [37]:
for var in binary_vars:
    full_biz[var] = 1*full_biz[var]
    
binary_df = full_biz[binary_vars].replace(np.nan, np.nan, regex=True)
binary_df.head()

,Caters,RestaurantsReservations,Ambience_romantic,Ambience_upscale,RestaurantsTakeOut,BikeParking,OutdoorSeating,BusinessParking_lot,HasTV,GoodForMeal_dinner,...,GoodForMeal_dessert,Ambience_touristy,BusinessParking_validated,GoodForMeal_lunch,RestaurantsGoodForGroups,GoodForKids,Ambience_casual,Ambience_divey,RestaurantsDelivery,Ambience_trendy
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,0.0,0.0


In [55]:
for var in text_vars:
    full_biz[var] = full_biz[var].replace(np.nan, '', regex=True)
    full_biz[var] = full_biz[var].astype(str)
    
text_df = full_biz[text_vars]
text_df.head()

,user_id,business_id,postal_code,city,categories,name,neighborhood,state
0,V2exkc2JwEdc8UNUNhT7sg,--6MefnULPED_I942VcFNA,L4B 3P7,Richmond Hill,"['Restaurants', 'Chinese']",John's Chinese BBQ Restaurant,nan,ON
1,j97y1ZAsYABIOcR0KQCy5Q,--6MefnULPED_I942VcFNA,L4B 3P7,Richmond Hill,"['Restaurants', 'Chinese']",John's Chinese BBQ Restaurant,nan,ON
2,8-KBKMJVO-j5YqpHb08j9w,--6MefnULPED_I942VcFNA,L4B 3P7,Richmond Hill,"['Restaurants', 'Chinese']",John's Chinese BBQ Restaurant,nan,ON
3,CxDOIDnH8gp9KXzpBHJYXw,--6MefnULPED_I942VcFNA,L4B 3P7,Richmond Hill,"['Restaurants', 'Chinese']",John's Chinese BBQ Restaurant,nan,ON
4,7Dykd1HolQx8mKPYhYDYSg,--6MefnULPED_I942VcFNA,L4B 3P7,Richmond Hill,"['Restaurants', 'Chinese']",John's Chinese BBQ Restaurant,nan,ON


In [59]:
for var in cat_vars:
    full_biz[var] = full_biz[var].replace(np.nan, '', regex=True)
    full_biz[var] = full_biz[var].astype(str)
    
cat_df = full_biz[cat_vars]
cat_df.head()

,Alcohol,WiFi,NoiseLevel,RestaurantsAttire
0,beer_and_wine,no,quiet,casual
1,beer_and_wine,no,quiet,casual
2,beer_and_wine,no,quiet,casual
3,beer_and_wine,no,quiet,casual
4,beer_and_wine,no,quiet,casual


In [60]:
cat_df.dtypes

Alcohol              object
WiFi                 object
NoiseLevel           object
RestaurantsAttire    object
dtype: object

In [62]:
for var in date_vars:
    full_biz[var] = full_biz[var].replace(np.nan, '', regex=True)
    full_biz[var] = full_biz[var].astype(str)
    
date_df = full_biz[date_vars]
date_df.head()

,date
0,2016-08-25
1,2010-12-06
2,2015-08-05
3,2010-12-22
4,2017-02-07


In [58]:
texts_df =  pd.DataFrame(index=['pbp'], columns=[text_vars], dtype=str)
text_df.dtypes

user_id         object
business_id     object
postal_code     object
city            object
categories      object
name            object
neighborhood    object
state           object
dtype: object

In [42]:
for var in float_vars:
    full_biz[var] = 1*full_biz[var]
    
float_df = full_biz[float_vars].replace(np.nan, np.nan, regex=True)
float_df.head()

,latitude,longitude,is_open,rating,stars,review_count,RestaurantsPriceRange2
0,43.840905,-79.399604,1,5,3.0,30,2.0
1,43.840905,-79.399604,1,1,3.0,30,2.0
2,43.840905,-79.399604,1,4,3.0,30,2.0
3,43.840905,-79.399604,1,2,3.0,30,2.0
4,43.840905,-79.399604,1,2,3.0,30,2.0


In [43]:
text_df.dtypes

user_id         object
business_id     object
postal_code     object
city            object
categories      object
name            object
neighborhood    object
state           object
dtype: object

In [64]:
full_biz2 = binary_df.join(text_df)
full_biz2 = full_biz2.join(float_df)
full_biz2 = full_biz2.join(cat_df)
full_biz2 = full_biz2.join(date_df)

In [66]:
full_biz2.dtypes

Caters                        float64
RestaurantsReservations       float64
Ambience_romantic             float64
Ambience_upscale              float64
RestaurantsTakeOut            float64
BikeParking                   float64
OutdoorSeating                float64
BusinessParking_lot           float64
HasTV                         float64
GoodForMeal_dinner            float64
BusinessParking_street        float64
GoodForMeal_breakfast         float64
BusinessParking_garage        float64
BusinessParking_valet         float64
WheelchairAccessible          float64
Ambience_intimate             float64
Ambience_hipster              float64
BusinessAcceptsCreditCards    float64
GoodForMeal_brunch            float64
Ambience_classy               float64
RestaurantsTableService       float64
GoodForMeal_latenight         float64
GoodForMeal_dessert           float64
Ambience_touristy             float64
BusinessParking_validated     float64
GoodForMeal_lunch             float64
RestaurantsG

In [ ]:
"""GoodForMeal_latenight          object
GoodForMeal_dessert            object
GoodForMeal_latenight          object
GoodForMeal_dessert            object
GoodForMeal_lunch              object
GoodForMeal_brunch             object
GoodForMeal_breakfast          object
GoodForMeal_dinner             object"""

In [ ]:
#full_biz.GoodForMeal_breakfast.unique()
mycat = str(full_biz.categories)

In [ ]:
#full_biz.dtypes

In [ ]:
uniques = list(full_biz.categories.unique())


In [ ]:
flat_list = [item for sublist in l for item in sublist]

In [ ]:
#flat_list = [item for sublist in l for item in sublist]

for sublist in uniques:
    for item in sublist:
        flat_list.append(item)

In [ ]:
def iterFlatten(root):
    if isinstance(root, (list, tuple)):
        for element in root:
            for e in iterFlatten(element):
                yield e
    else:
        yield root

In [ ]:
#list(iterFlatten(uniques))

In [ ]:
import itertools
#list_of_cats = list(itertools.chain.from_iterable(uniques))
#list_of_cats = sum(uniques, [])
list_of_cats = []
map(list_of_cats.extend, uniques)

In [ ]:
list_of_cats

In [ ]:
""float_vars = ['stars','latitude','RestaurantsPriceRange2','longitude']
int_vars = ['is_open','review_count']""

In [ ]:
test_vars = list(full_biz.columns)
#test_vars

In [ ]:
test_vars

In [ ]:
full_biz2.dtypes

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1,1, figsize =(10,8))
sns.heatmap(full_biz2.corr(), ax=ax)

In [ ]:
#dropping based on collinearity

#We have relatively few collinear points although we do have quite a few missing values - this is something 
#we'll discover soon enough in later EDA. 



In [ ]:
#full_biz2.dtypes

In [ ]:


full_biz2.WiFi = full_biz2.WiFi.replace(np.nan, '', regex=True)
full_biz2.WiFi.unique()

In [ ]:
object_to_text_vars = ['user_id', 'business_id', 'postal_code','city','Alcohol','categories','name',
                       'WiFi','NoiseLevel','state','date']

for var in object_to_text_vars:
    full_biz2[var] = full_biz2[var].replace(np.nan, '', regex=True)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 8))
plt.scatter( full_biz2.rating, full_biz2.GoodForMeal_breakfast, color=sns.color_palette(), alpha=0.75)
plt.xlabel("User's Average Rating")
plt.ylabel('Number of Reviews')
plt.title('Average Rating by User Activity') 

In [ ]:
good_for = ['Good']

In [ ]:
#plt.scatter(full_biz2.rating, full_biz2.GoodForMeal_breakfast)



x_filt = full_biz2.rating[GoodForMeal_breakfast>0]
y_filt = full_biz2.GoodForMeal_breakfast[GoodForMeal_breakfast>0]

plt.scatter(x_filt, y_filt)

In [ ]:
"""full_biz2['new'] = full_biz2['user_id'].str.split(',') 
#df['column_new'] = df['column'].str.split(',') 
full_biz2.dtypes"""

In [ ]:
full_biz2.state.unique()

In [ ]:
tips.columns = ['business_id', 'user_id', 'unidentifiedinfo_tips'] 

tips['business_id'] = tips['business_id'].map(lambda x: x[1:])
tips['user_id'] = tips['user_id'].map(lambda x: x[:-1])
tips.head()

In [ ]:
users.head()

In [ ]:
"""
Based on this knowledge, the following EDA steps may be necessary:

1. Label column headers - 
    a. Check-ins: User_ID, # of Check-ins
    b. Photos: User_ID, # of Photos
    c. Review: Business_ID?, User_ID?, Stars, Date (of the review?)
    d. Biz: Use column headers from dataset
    e. Tips: Business_ID, User_ID, # of tips?
    
    Questions that arise:
    - We have a wide range in the number of records across sets. Is it because 
    of the number of businesses, number of users, or both? We should confirm
    the population of each set and make sure we're clear on that in this EDA.
    
2. Merge variables as needed - 
    a. It may be easiest to work with one large dataset that is at the user-level
    that contains all of the information that we need. If so, the following pieces
    may be necessary:
    b. Determine what level we should be working at - user, review, or business
        i. Seems like we should be at the user-level, in which case, will we have 
        multiple duplicates for the users depending on how many reviews we have?
    c. Merge depending on level determined.
    
    Questions that arise:
    - 
    
3. Basic cleaning - 
    a. Do we have duplicates in any of these datasets? If so, why?
    b. Missing values - 
        i. Due to that merging, we should have missing values in some users
        ii. Figure out a strategy for dealing with those missing values. 
    
4. Do basic EDA - 

    a. How many unique users? Businesses? Reviews? 
    b. User behavior - 
        i. What do the most and least active users look like? Are they enthusiastic?
            Do they travel? 
        ii. Do a user's reviews tend to favor a particular style of restaurant 
            (is there a clustering of taste inherent in the data already)?
        iii. What do reviews look like? Are there certain variables that are 
            are always missing? 
    
    Questions that arise:
        - Do we have enough information about folks to adequately make these predictions?
            What else might we need to impute?
            
